<a href="https://colab.research.google.com/github/fur-elyse/Cartoonization-AI/blob/master/data_collection/colab_tara_step2a_facebook_api_data_pull_bm_08242020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
## Install Libraries

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144800 sha256=db3c2629a780e3db13334b53f89bba1bff5cbc12db73ab83c3d9dfcee5c69290
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import requests
import time
import json
import re
from pandas import json_normalize
from fuzzywuzzy import fuzz

## References

In [ ]:
# # https://developers.facebook.com/docs/graph-api/reference/place-information/
# https://developers.facebook.com/docs/places/web/search
# https://www.pythonforbeginners.com/requests/using-requests-in-python/

#App Dashboard
# https://developers.facebook.com/apps/294212045198068/dashboard/

# Graph Explorer
# https://developers.facebook.com/tools/explorer/1171039436612818/

# Graph API Rate Limits
# https://developers.facebook.com/docs/graph-api/overview/rate-limiting#application-level-rate-limiting

# App Review
# https://developers.facebook.com/docs/app-review#business-verification

# Permission References
# https://developers.facebook.com/docs/permissions/reference


# https://developers.facebook.com/docs/messenger-platform/reference/send-api/error-codes

# PLACE API
# https://developers.facebook.com/docs/graph-api/reference/place-information/

## CURL

In [ ]:
# ACCESS_TOKEN = "EAAELlZARxUvQBAHO2OCDkmIlcvRMOZBVY9HESfElSJNDKF4ZBLeXUf03NtV4Yc13VRh9lmmPgQcZBlMk82NS6vgRAkVCwlchDsKwDP1mljn813neswEYZBCeE9fQEEKyNhZA5aTIJm1X0P15mkY2OWzVoOcmX4pCUbUOZC4EteUE7H5qB722AOGQtFgZBglaZArSsC6AlxTrHRTIkeDGAJfAx"
# ACCESS_TOKEN

In [ ]:
# !curl -X GET "https://graph.facebook.com/search?type=place&q=McDonalds&fields=name,checkins,website&access_token={ACCESS_TOKEN}"

In [ ]:
# !curl -X GET "https://graph.facebook.com/search?type=place&center=37.4683,-122.1439&fields=name,checkins,website&access_token={ACCESS_TOKEN}"

In [ ]:
# La Union	San Juan	Resort	Resort (Generic)	San Juan Surf Resort	sanjuansurf@yahoo.com.ph	09178875470	Richie Marie L. Bermudez	16.6570663	120.3201828				OPERATIONAL	(072) 687 9990	{'location': {'lat': Decimal('16.6570663'), 'lng': Decimal('120.3201828')}, 'viewport': {'northeast': {'lat': Decimal('16.65845118029151'), 'lng': Decimal('120.3213688302915')}, 'southwest': {'lat': Decimal('16.6557532197085'), 'lng': Decimal('120.3186708697085')}}}	https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png	ChIJV4TZruqNkTMRF01rJyImLAQ		[{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107488884706282490917">Marti Zurriel Mercado</a>'], 'photo_reference': 'CmRaAAAA3ZVMhPU9razCSxEEgLXGeG3PQ8PPyZD0U0WH34cIyF4DxRYYGGCV70JjQJPYk1eLQ-rlarzD_gOjxFPoqYOIiIE4OBkHw826suVncRb28u3gSR9dBefrqU2D_5ksEj0MEhDifa9_4coGvoU97_SN8_SdGhTJ811nJwVXD7uY7jTZ4TK6YUSMMQ', 'width': 4032}, {'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107488884706282490917">Marti Zurriel Mercado</a>'], 'photo_reference': 'CmRaAAAAfMtgV6FcrzMbypiEbt34Gv2FCjtC238PeKs5M5VHOUxDZ8OuaMFX-kmd8E_mKEWQ4RKE8xtq45X7HyzPqc-iOE7a-gnxHg-3T9YKGdnVKzYBHV5NdiQ-tv61sSdqmOjMEhBL7KKbot7v3GnW7ospwv4FGhSj2Yn8P8kgxMmjnt-oHpx2eoSnpw', 'width': 4032}, {'height': 3096, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104032837938043101662">Cristiano Esclapon</a>'], 'photo_reference': 'CmRaAAAA-aUEp9gGEEdi-FIcZ-ybOeOI4wx2P2nZZI9KVA-iSU6uLooLbMYfE8cDJ4KPsVSii4I_W0HwNa3oEtAZqCHIeceAii7MAeA6rCA3SoM6p1XEHrPQgUuP7cok3YOJ7PXIEhD710OLVjrTWdEdSg51KUdeGhTKZpWtMmqpVEbvwVk1jAkXJ8YAKw', 'width': 4128}, {'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116101372980511595292">Jogi Manalo</a>'], 'photo_reference': 'CmRaAAAAOHX9HZ1Nxq_vHAVJgz2RqPmWAGGg5G2K2AD5xLfbZvxwcFYYrXi_eyWdPmT77eUDZZ6sNmKSFyhjGv23yZUaf5nvxbL6ftq1i1sv-1zQxfMTGVzw_43NaP51E1P778fMEhDePB5p_tbV-aGpvYrVYqr_GhT_JWIvhl7A1SQ4bBF9y76xtla3zw', 'width': 1920}, {'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117783923857292196022">Benedict Carreon</a>'], 'photo_reference': 'CmRaAAAAS-QF4l5zge_9OPiC64FUdlBNg6eVGxPvAowqBIMNNfYcuv522aJIO5xh-MmGbKKsJAmkIEFuHXTpSHf1m9sHRZXgHjVSeEQGgo2ZY7O7G8-Rm4hkO7QgdOpYZ7xLwLkVEhCrMbvjyIzhM7yGVaUedCreGhR1YpRd1B3Y8J4GmypqnThI8y9uVQ', 'width': 5184}, {'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117783923857292196022">Benedict Carreon</a>'], 'photo_reference': 'CmRaAAAAaVSUaaQoEPgqV9d9mgwAREtOgeShzDO95QBfBqL2OJuizcsKzuc4_j4y9EexbrsMUV8BVjfHWC37vh8QY6TULUJhWl_noMO9CXv0PfFT-6GtYqZ8h-YjoLJgz7j13pAzEhC-NMal_YMV9RCS-EOlbZbZGhTHQ_6BVD-_OFZbjl2onflcnle1tg', 'width': 5184}, {'height': 2736, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117665111349375787636">Dexter Adr</a>'], 'photo_reference': 'CmRaAAAAYIyJDcA6Y3dmKMf19mQCJkjmDQWhe_W3GFYndR_t4nX0ppoumS_bse7V7f6MPWezFGKVR6JfXy2FmcPWZ3dSPYQgK2DhrVGCo92Ov26AIlJmls4XRiIsQpknN9em5VPsEhC-N2d7yaPPc-9Fidqs-OQkGhRhtoa-VzUoOX7dYlmLAljMAE5oog', 'width': 3648}, {'height': 3096, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109745086732774679874">Joey Principe</a>'], 'photo_reference': 'CmRaAAAATBkhh_Yi8VV8PgOtZUtGv7IaDstVUV72LEa_lIJlRj1ZKCrGakWIzKOIj2Fcc_FvsbS9VlGtveOGiCgQNNOEWaAPguCcEbdmHTRAljf5SqGVXK4LVHPuk87zsBZwWASAEhDmT9iBsWPbKU2nn1RSN_xjGhTI9BsV3o7r47V1zlG3D0PlFXwmjw', 'width': 4128}, {'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104393754686284832478">Teejay T.</a>'], 'photo_reference': 'CmRaAAAAAcqZV1sY0_tMK9x4qwGv1vhjhx76uqb4i35coMqmmWphuVWC4P_vTXGHj3GlT_m0ROQVaiMZhDp-hBDcPVa4VADHV0_svNArrarJ32dWIPyLo6p0zVl54_hUgrZ6r0JJEhDi4GinRliIX_1u09uCS3tCGhSgD_9dTZrYnCWHsefEaAzITAGunQ', 'width': 4032}, {'height': 2976, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105529653109495971929">Edna Merioles</a>'], 'photo_reference': 'CmRaAAAAmEhzbQcTbhrxxCJxRM37s0tQJ0VqxmEbSTR7woawT-o3dxjs6QiObfpVXi0RnE4LKci9ytbuiYK_NgRirFycD7yXbZqqtz6DmXzJ5BFrRai1GQydPLhcOZgVsuO7df1IEhBquIWGm-_grFkFELMiTZMJGhQvlYuF-q43gU9PruVPLiIoMkO7mg', 'width': 3968}]	4.3	[{'author_name': 'Stephen Salazar', 'author_url': 'https://www.google.com/maps/contrib/113128590046867398073/reviews', 'language': 'en', 'profile_photo_url': 'https://lh3.googleusercontent.com/a-/AOh14GhG_0wiBBeay9vGv4YaajxQG4MvARDNgyPmkwEIaPg=s128-c0x00000000-cc-rp-mo-ba5', 'rating': 4, 'relative_time_description': '5 months ago', 'text': "Went there so my 14 y/o son could try out surfing. he had a blast and wants to surf again.\n\nBeginners lesson was what we registered for and\nP500 per 1 hour is pretty reasonable because their surf coaches were good and knew what they were doing. they even extended to 6 pm with no charge, because they were really into it...\n\nwaiting for the waves and launching students at the right time is a skill.\n\nThe best time to go there is at 4 pm because it's high tide.\nBetter to leave your car at your hotel cause parking could get tricky. we were lucky to get a spot right next to the resort. \n\nIt's also a good time to wait for the sunset over there. it's really breath taking.\n\nThanks kuya Surf dude for teaching my boy!!!\n\nSurf's up!", 'time': 1583374813}, {'author_name': 'John Carlo Querubin', 'author_url': 'https://www.google.com/maps/contrib/112568826046509674672/reviews', 'language': 'en', 'profile_photo_url': 'https://lh3.googleusercontent.com/a-/AOh14GjCDYx3ENp5jdr2LX-mY3y43vWMnhPX4J5TELcP0Q=s128-c0x00000000-cc-rp-mo-ba3', 'rating': 4, 'relative_time_description': '5 months ago', 'text': "We got the smallest room for P2000 (Feb 2020). It was cheaper than the hostel we stayed in at 2800 for 2.\n\nThe room wasn't great as expected but it should have at least been designed as a liveable space. It was a bed, a table, a toilet, a sink, a shower head and air-conditioning. That's it. Good enough for sleeping a night or two.", 'time': 1582680435}, {'author_name': 'Joshua Dimapilis', 'author_url': 'https://www.google.com/maps/contrib/107210808842146332826/reviews', 'language': 'en', 'profile_photo_url': 'https://lh3.googleusercontent.com/a-/AOh14Gjd9fg8hIMPEGypcUleeal5UZR8T-7AElig8KNJtQ=s128-c0x00000000-cc-rp-mo', 'rating': 4, 'relative_time_description': '7 months ago', 'text': 'Food was okay and good thing they have tea though its not on the menu. Place feels like a relaxing resort with the lack of swimming pool. Good thing is that they accept credit cards and debit cards which so convenient.', 'time': 1578207619}, {'author_name': 'paul erichson', 'author_url': 'https://www.google.com/maps/contrib/109969462196361891941/reviews', 'language': 'en', 'profile_photo_url': 'https://lh3.googleusercontent.com/a-/AOh14GgDhojmNuo6de7CYDQ1V8vLJHNndI534y_hXq0PsA=s128-c0x00000000-cc-rp-mo-ba2', 'rating': 4, 'relative_time_description': '7 months ago', 'text': 'Great resort for surfing. Breakfast was free. WiFi was OK. But just a heads up the restaurant is average but there are a lot of great alternative places around.\n\nAlso it cost 600 peso to surf for half a day if anyone was wondering', 'time': 1577521035}, {'author_name': 'Kester. BT', 'author_url': 'https://www.google.com/maps/contrib/106988861361980917438/reviews', 'language': 'en', 'profile_photo_url': 'https://lh3.googleusercontent.com/a-/AOh14Ghpyas2-Y8xL6NF2tk6nxkXmcwvlgqP5EM7jIJtxg=s128-c0x00000000-cc-rp-mo-ba4', 'rating': 5, 'relative_time_description': '7 months ago', 'text': 'One of the best place to stay in San Juan La Union. 😊 ( My family always love to stay here )\n\nCoastcall always serves good food. \n\nCustomer service IMPROVED. Nice! 😉 \n\nTIP❗ BOOK EARLY! 😉', 'time': 1578226744}]	['lodging', 'point_of_interest', 'establishment']	https://maps.google.com/?cid=300657203259067671	1310	Urbiztondo, San Juan, MacArthur Highway, San Fernando

In [ ]:
# !curl -X GET "https://graph.facebook.com/search?type=place&center=16.6570663,120.3201828&fields=name,checkins,website&access_token={ACCESS_TOKEN}"

# Requests Final

## EDA Source CSV

### Read csv file

In [ ]:
# src_fname_type = "f1" 
# src_fname = "List_of Accredited AEs-August_8_2020_c$_master_data_w_map_details.csv"
# src_fpath = "/content/drive/My Drive/DOT/" + src_fname

src_fname_type = "f2" 
src_fname = "DoT_Region 1_06082020_old_featur.csv"
src_fpath = "/content/drive/My Drive/DOT/" + src_fname

src_df = pd.read_csv(src_fpath)

In [ ]:
src_df.head()

,Category,Entity Name,Province,City/Municipality,General Manager,Phone/Mobile Number,Email Address,Full Address,business_status,formatted_address,formatted_phone_number,geometry,icon,place_id,international_phone_number,name,open_hours,photos,rating,reviews,place_types,map_url,num_ratings,vicinity
0,Health and Wellness\nServices,Dagupan City Spa,Pangasinan,Dagupan,Ana Cristina Sinlao,075-523-4030,dagupan_cityspa@yahoo.co m,"Dagupan City Spa, Dagupan, Pangasinan",OPERATIONAL,"CSI The City Mall Dagupan - Binmaley Rd, Dagup...",0909 114 3279,"{'location': {'lat': Decimal('16.0238385'), 'l...",https://maps.gstatic.com/mapfiles/place_api/ic...,ChIJkfGUdz9dkTMRxNCdg6056-8,+63 909 114 3279,Flawless Face and Body Clinic - CSI The City M...,"{'open_now': True, 'periods': [{'close': {'day...","[{'height': 787, 'html_attributions': ['<a hre...",4.9,"[{'author_name': 'Ruby Vinluan', 'author_url':...","['beauty_salon', 'hair_care', 'doctor', 'spa',...",https://maps.google.com/?cid=17287975011992064196,8.0,"CSI The City Mall Dagupan - Binmaley Road, Dag..."
1,Health and Wellness\nServices,"Lorma Medical Center, Inc.",La Union,San Fernando,Robert F. Kaiser,(072) 700-0000,medicalinquiry@lorma.org,"Lorma Medical Center, Inc., San Fernando, La U...",OPERATIONAL,"MacArthur Highway, San Fernando, 2500 La Union...",(072) 700 0000,"{'location': {'lat': Decimal('16.6315149'), 'l...",https://maps.gstatic.com/mapfiles/place_api/ic...,ChIJwblP1t-NkTMRggEeFwRrMMQ,+63 72 700 0000,Lorma Medical Center,"{'open_now': True, 'periods': [{'open': {'day'...","[{'height': 1456, 'html_attributions': ['<a hr...",3.9,"[{'author_name': 'Oskie Ravanera', 'author_url...","['hospital', 'health', 'point_of_interest', 'e...",https://maps.google.com/?cid=14136916895627870594,32.0,"MacArthur Highway, San Fernando"
2,Mount Balungao Magnesium Transdermal Spa,Mel Nature's Spa and Salon,La Union,AGOO,Meldea F. Solomon,9208214376,melnaturespa@yahoo.com,"Mel Nature's Spa and Salon, AGOO, La Union",OPERATIONAL,"Dr FP Pacis Building, consolacion, MacArthur H...",0920 821 4376,"{'location': {'lat': Decimal('16.3247708'), 'l...",https://maps.gstatic.com/mapfiles/place_api/ic...,ChIJGW3MSN93kTMRV0ibxgFAkaY,+63 920 821 4376,Mel Nature's Spa & Salon,"{'open_now': True, 'periods': [{'close': {'day...","[{'height': 960, 'html_attributions': ['<a hre...",5.0,"[{'author_name': 'Khayzie Rivera Nidoy', 'auth...","['beauty_salon', 'point_of_interest', 'establi...",https://maps.google.com/?cid=12002444858290292823,1.0,"Dr FP Pacis Building, consolacion, MacArthur H..."
3,Health and Wellness Services,NaN,Pangasinan,Balungao,Angelito D. Uminga,(075) 636- 5337/09167314890/0998248\n2300,mountbalungaospa@gmail.co m,", Balungao, Pangasinan",NaN,"Balungao, Pangasinan, Philippines",NaN,"{'location': {'lat': Decimal('15.8837087'), 'l...",https://maps.gstatic.com/mapfiles/place_api/ic...,ChIJLR-Qo0g7kTMRlQzAmzOi7sA,NaN,Balungao,NaN,"[{'height': 1987, 'html_attributions': ['<a hr...",NaN,NaN,"['locality', 'political']","https://maps.google.com/?q=Balungao,+Pangasina...",NaN,Balungao
4,Health and Wellness\nServices,Rivera Foot and Body Massage (Nuat Thai Agoo),La Union,AGOO,Baby Ruth A. Campay,(072) 888-\n0471/09270733097,Nuatthaiagoo@gmail.com,"Rivera Foot and Body Massage (Nuat Thai Agoo),...",OPERATIONAL,"Agoo, La Union, Philippines",NaN,"{'location': {'lat': Decimal('16.3272385'), 'l...",https://maps.gstatic.com/mapfiles/place_api/ic...,ChIJNSpUjGx3kTMRox6zp0eIrkA,NaN,Nuat Thai,"{'open_now': True, 'periods': [{'close': {'day...","[{'height': 405, 'html_attributions': ['<a hre...",NaN,NaN,"['spa', 'point_of_interest', 'establishment']",https://maps.google.com/?cid=4660812505712631459,NaN,Agoo


In [ ]:
# src_df.info()

In [ ]:
# src_df['Entity Name'].value_counts()
src_df['Entity Name'].isnull().value_counts()

False    869
True       1
Name: Entity Name, dtype: int64

In [ ]:
filtered_src_df = src_df[src_df['Entity Name'].notnull()]

In [ ]:
filtered_src_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 869 entries, 0 to 869
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Category                    869 non-null    object 
 1   Entity Name                 869 non-null    object 
 2   Province                    869 non-null    object 
 3   City/Municipality           869 non-null    object 
 4   General Manager             865 non-null    object 
 5   Phone/Mobile Number         867 non-null    object 
 6   Email Address               861 non-null    object 
 7   Full Address                869 non-null    object 
 8   business_status             614 non-null    object 
 9   formatted_address           619 non-null    object 
 10  formatted_phone_number      514 non-null    object 
 11  geometry                    619 non-null    object 
 12  icon                        619 non-null    object 
 13  place_id                    619 non

### Preprocess long lat 


In [ ]:
# import json
# import re

# t = src_df['geometry'][0]
# loc_data = re.findall(r'[+-]?[0-9]+\.[0-9]+', t)
# latitude = loc_data[0]
# longitude = loc_data[0]

## Pull data from FB Graph API

In [ ]:
# API Rate Limit 200/Hr per App
# https://developers.facebook.com/docs/graph-api/overview/rate-limiting

### Get fb json data fcn

In [ ]:
def get_fb_json_data(access_token=None, latitude=None, longitude=None, distance_val=None, place_name=None, included_fields=None):
    
    if place_name != None:
        target_url = f"https://graph.facebook.com/search?type=place&q={place_name}&fields={included_fields}&access_token={ACCESS_TOKEN}"
        
    if distance_val != None:
        target_url = f"https://graph.facebook.com/search?type=place&center={latitude},{longitude}&distance={distance_val}&fields={included_fields}&access_token={ACCESS_TOKEN}"
        
    if place_name == None and distance_val == None:
        target_url = f"https://graph.facebook.com/search?type=place&center={latitude},{longitude}&fields={included_fields}&access_token={access_token}"
        

    # https://graph.facebook.com/search?type=place&center=16.6570663,120.3201828&fields=name,checkins,website&access_token={ACCESS_TOKEN}

    r = requests.get(target_url)

    # === pretty printing data
    # pretty_data = json.dumps(r.json(), indent=4)
    # print(type(pretty_data))
    # print(pretty_data)

    # === dictionary type for parsing
    json_data = r.json()
    # print(json_data)
    # print(json_data['data'][0]['id'])
    # print(pretty_data["data"])

    return json_data

### Rerun from here - Get file timestamp

In [ ]:
# get file timestamp
timestamp = str(dt.now())
for r in ((":", "_"), (" ", "_")):
    timestamp = timestamp.replace(*r)

timestamp = timestamp.split(".")[0]

### Initialize params

In [ ]:
#Travel Flutter
ACCESS_TOKEN="EAAELlZARxUvQBAFGhuy2Air9ePPeF1lTaliZAyUajLeZCo0SZApgbZAcwcHDApUFlYTrP5IZBcYg9tlm6UkRZBnszLRefNY2iGSHyyCwmDJxgQYhoW7ZAriSsGAuAGWFBOnqH8KAWVr4MQYymjmdCNVkOcjZC1VK9yj4NEoOZCw5cJkcfobGmFz8FltXlcACofHD34ZC5ZA8c9BfZBDZCnzevoN8gT"

#Travel Hr Admin
# ACCESS_TOKEN="EAAELlZARxUvQBAAEEvpjc5WppEPn3GYSlT8iQ7gJ8xyZBS9SWJrLZCb5j6hnJFZAmDMZBBwwcxlhbWN5MGCZBd3md0j5wKs10EXIjHnAsQzYFeZCvRWnFdCHKKS2HPaBNOL7RbTBLFSvQxn0UwHZCZA5yD4FLrNQXoTCpaGdimfjLZAeQv8OAG6HCSHR9BYNQgIjMesmbM5aIF9gZDZD"

#initialize here src_df
df = src_df.copy()

# distance_val = 10
# col_name_main = "fb_data_json_not_parsed" + str(distance_val)
col_name_main = "fb_data_json_not_parsed"

#Do this as a post-process
# col_name_len = "fb_data_q_len"+ str(distance_val)
# df[col_name_len] = None
df[col_name_main] = None

add_columns = ["about", "category_list", "checkins", "name1", "cover", "description", "differently_open_offerings", "engagement", "link", "location", "name", "parking", "payment_options", "price_range", "rating_count", "restaurant_services", "restaurant_specialties", "temporary_status", "website"]
for new_col in add_columns:
    df[new_col]= None

included_fields = "id, about, category_list, checkins, cover, description, differently_open_offerings, engagement, link, location, name, parking, payment_options, price_range, rating_count, restaurant_services, restaurant_specialties, temporary_status, website"

### Extract JSON

In [ ]:
"""Extract nested values from a JSON tree."""


def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

### Pulling FB Data

#### Batch API calls bc API rate limit


##### f1 (file1)
src_fpath = "/content/drive/My Drive/DOT/List_of Accredited AEs-August_8_2020_c$_master_data_w_map_details.csv"

###### place_name_query

In [ ]:
# place_name_query
# 0 - 235
# 235 - 430
# 240 - 479
# 479 - 545


##### f2 (file2)
src_fpath = "/content/drive/My Drive/DOT/DoT_Region 1_06082020_old_featur.csv"

In [ ]:
# place_name_query
# 0 - 243
# 243 - 470
# 470 - 631
# 631 - 869 
    

###### place_name_query

In [ ]:
# place_name_query


#### API Request fcn per se

In [ ]:
#optimize in the future, look for map equi
for index, row in df.iterrows():
    # time.sleep(1)

    # if f1:
    # if index >= 479 and index <=545:

    # if f2
    if index >= 631 and index <= 870:
    # if index >= 765 and index <= 870:
        print("=====")
        print("=====")
        print(f"index:{index}")
        print("\n")

        # using place_name
        if (row['Entity Name'] is np.nan) == False:

            place_name = row['Entity Name']
            query_type = "q_name"
            fb_json_data = get_fb_json_data(access_token=ACCESS_TOKEN, place_name=place_name, included_fields=included_fields)
            fb_json_data_normalized = json_normalize(fb_json_data, sep="_")

            fpath = "/content/drive/My Drive/DOT/fb_data_api/"
            json_fname = str(index) + "_" + src_fname_type + "_" + query_type + "_" + timestamp

            q_dict = {"about": "", 
                      "category_list": "", 
                      "checkins": "",
                      "name1": "",
                      "cover": "",
                      "description": "",
                      "differently_open_offerings" : "",
                      "engagement": "",
                      "link": "",
                      "location": "",
                      "name": "",
                      "parking": "",
                      "payment_options": "",
                      "price_range": "",
                      "rating_count": "",
                      "restaurant_services" : "",
                      "restaurant_specialties" : "",
                      "temporary_status": "",
                      "website" : ""
                      }

            print("fb_json_data_print", fb_json_data)
    
            try:
                result_ctr=1
                print("=====")
                print("Iterating through the json results...")    
                for result in fb_json_data['data']:
                    print(f"Result {result_ctr}")
                    name = json_extract(result, 'name')

                    match_found = False
                    for name_elem in name:
                        
                        str1 = row['Entity Name'].lower()
                        str2 = name_elem.lower()
                        fuzz_ratio = fuzz.ratio(str1,str2)

                        if fuzz_ratio >= 90:
                        # if(row['Entity Name'] == name_elem):
                            match_found = True
                            print("\n*****")
                            print("It's a match!")
                            print(f"row['Entity Name']: {str1}")
                            print(f"name_elem: {str2}")
                            print(f"fuzz_ratio: {fuzz_ratio}")

                            # fb_data_fields            
                            field_cols = ["about", "category_list", "checkins", "name1", "cover", "description", "differently_open_offerings", "engagement", "link", "location", "name", "parking", "payment_options", "price_range", "rating_count", "restaurant_services", "restaurant_specialties", "temporary_status", "website"]
                            print("Entity Name:", row['Entity Name'])

                            print("=====")
                            for field in field_cols:
                                print(f"field: {field}")

                                try:
                                    df[field][index] = result[field]
                                    print(f"{field} : {result[field]}")
                                except Exception as e:
                                    print("=====")
                                    print(f"Exception occurred: {e}")
                                    df[field][index] = ""

                                    continue
                            print("\n")

                        if match_found == True:
                            break

                    result_ctr=result_ctr+1
                        
                if match_found == False:
                    print(f"\nNo match found for Entity Name:  {row['Entity Name']}")     

            except Exception as e:
                print("!!!!!!")
                print("Exception occurred")
                print(e)

=====
=====
index:631


fb_json_data_print {'data': [{'id': '696323100985396', 'about': 'Te amos Dios', 'category_list': [{'id': '1602', 'name': 'Public Figure'}], 'checkins': 0, 'cover': {'cover_id': '701718233779216', 'offset_x': 50, 'offset_y': 50, 'source': 'https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-9/s720x720/117646599_701718240445882_5524467953331681661_o.png?_nc_cat=107&_nc_sid=dd9801&_nc_ohc=kl8LS1nt3dcAX-4BlnW&_nc_ht=scontent-atl3-1.xx&oh=45c8d4406a7e22cee16d2d81932047b7&oe=5F67A33E', 'id': '701718233779216'}, 'engagement': {'count': 4558, 'social_sentence': '4.5K people like this.'}, 'link': 'https://www.facebook.com/696169631000743', 'location': {'city': 'La Caleta', 'country': 'Dominican Republic', 'latitude': 18.45332, 'longitude': -69.66845, 'street': '18293236507', 'zip': '84923'}, 'name': 'Joselito Lopez', 'rating_count': 0}, {'id': '1838421113060316', 'about': 'Joselito Casa de Comidas is an intimate Spanish restaurant reminiscent of the quintessential establishments

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.
name_elem: kristahl holidays inc.
fuzz_ratio: 100
Entity Name: Kristahl Holidays Inc.
=====
field: about
about : Kristahl Holidays Inc.
field: category_list
category_list : [{'id': '162914327091136', 'name': 'Travel Agency'}]
field: checkins
checkins : 3
field: name1
=====
Exception occurred: 'name1'
field: cover
cover : {'cover_id': '1293461090703444', 'offset_x': 48, 'offset_y': 50, 'source': 'https://scontent-atl3-1.xx.fbcdn.net/v/t1.0-9/s720x720/17553937_1293461090703444_206678918911920688_n.jpg?_nc_cat=100&_nc_sid=dd9801&_nc_ohc=uG0avLqGyN0AX_u5KEx&_nc_ht=scontent-atl3-1.xx&_nc_tp=7&oh=814fbd2f92f0ec72de846d100ad2e8ae&oe=5F67AEBF', 'id': '1293461090703444'}
field: description
description : Engage in, conduct and carry on the business of a travel and tours agency, to arrange international and domestic ticketing travel services, to present tour operators in the Philippines and in other countries; to manage, supervise, and provide al

### Save df to csv in GDrive

In [ ]:
#df to csv
fpath = "/content/drive/My Drive/DOT/fb_data_api/"
fname = col_name_main + "_" + src_fname_type + "_" + query_type + "_" + timestamp
df.to_csv(fpath+fname+".csv", index=False)

# JSON Parsing Resources

In [ ]:
# https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e
# https://towardsdatascience.com/how-to-parse-json-data-with-python-pandas-f84fbd0b1025
# https://towardsdatascience.com/how-to-flatten-deeply-nested-json-objects-in-non-recursive-elegant-python-55f96533103d
# https://hackersandslackers.com/extract-data-from-complex-json-python/
# https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
# https://datatofish.com/export-pandas-dataframe-json/
# https://hackersandslackers.com/json-into-pandas-dataframes/